In [ ]:
## Uncomment command below to kill current job:
#!neuro kill $(hostname)

In [ ]:
# reload modules automatically:
%load_ext autoreload
%autoreload 2

# make user code available:
import sys
from pathlib import Path
module_path = str(Path('../src').absolute())
if module_path not in sys.path:
    sys.path.insert(0, module_path)

from dataset import split_dataset, BoneAgeDataset, normalize_target
from transforms import get_transform
from model import m46, convert_checkpoint
from train import get_parser, main
from const import DATA_PATH, MODELS_DIR, ROOT_PATH

import torch
from torch.utils.data import DataLoader
from catalyst.dl.runner import SupervisedRunner

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import islice
from IPython.core.display import Image, display

## Download the dataset

For demo purposes, we use a tiny version of the dataset with only 500 out of 12k train images and 20 out of 200 test images.

You can find the full dataset at http://data.neu.ro/bone-age-full.zip

In [ ]:
! [ ! -f /tmp/bone-age.zip ] && wget http://data.neu.ro/bone-age-tiny.zip -O /tmp/bone-age.zip

In [ ]:
! unzip -o /tmp/bone-age.zip -d /tmp/bone-age

In [ ]:
DATA_PATH = Path('/tmp/bone-age/data')
MODELS_DIR = Path('/tmp/bone-age/models')

list(DATA_PATH.iterdir()), list(MODELS_DIR.iterdir())

In [ ]:
bone_age_frame = pd.read_csv(DATA_PATH / 'train.csv') # ground truth
image_root = DATA_PATH / 'train' # radiographs root

In [ ]:
bone_age_frame

### Analyze gender distribution

In [ ]:
bone_age_frame['boneage_years'] = bone_age_frame['boneage'] / 12
fig = plt.figure(figsize=(18, 8))
for i, gender in enumerate(['male', 'female'], 1):
    ax = plt.subplot(1, 2, i)
    bone_age_frame.loc[bone_age_frame['male'] == (gender == 'male')].hist('boneage_years', ax=ax)
    ax.set_title(f'{gender} cohort', fontsize=22)
    ax.tick_params(axis='x', labelsize=16)
    ax.tick_params(axis='y', labelsize=16)
plt.tight_layout()
ax = fig.add_subplot(111, frameon=False)
ax.set_title('Bone age distribution', pad=30, fontsize=24)
ax.set_xlabel('years', fontsize=20)
plt.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')

### Plot some radiographs

In [ ]:
def plot_radiographs(dataset, nimages, predictions=None):
    ncols = 4
    nrows = int(np.ceil(nimages / ncols))
    fig = plt.figure()
    for i, sample in enumerate(islice(dataset, nimages), 1):
        image, label, img_id = sample['image'], sample['label'], sample['id']
        if torch.is_tensor(image):
            image = np.squeeze(image.numpy())
            label = normalize_target(label.item(), reverse_norm=True)

        ax = plt.subplot(nrows, ncols, i)
        if predictions is not None:
            ax.set_title(f'id {img_id}, true {label:n}, pred {int(predictions[i - 1]):n}', fontsize=24)
        else:
            ax.set_title(f'id {img_id}, {label:n} months', fontsize=24)
        ax.axis('off')
        ax.imshow(image, cmap='Greys_r')

    figsize = 6
    aspect_ratio = image.shape[0] / image.shape[1]
    fig.set_figheight(aspect_ratio * nrows * figsize)
    fig.set_figwidth(ncols * figsize)
    plt.tight_layout()
    
    ax = fig.add_subplot(111, frameon=False)
    crop_sz = 'x'.join(map(str, image.shape))
    ax.set_title(f'Crop size {crop_sz}', pad=40, fontsize=28)
    plt.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
    plt.pause(0.001)
    plt.show()

In [ ]:
# the preprocessed data is scaled to 2080x1600 (HxW)
boneage_dataset = BoneAgeDataset(bone_age_frame=bone_age_frame, root=image_root, transform=None)

nimages = 8
plot_radiographs(boneage_dataset, nimages)

### Familiarize yourself with dataset augmentation

In [ ]:
display(Image('1396_crop_area.png', width=400, unconfined=True))

In [ ]:
# With `get_transform` function we extract cropped, rescaled and augmented regions of interest
# This allows us to experiment with different areas of a radiograph. See the paper referenced in README
# We also normalize labels by demeaning and rescaling to (120, 120), see `normalize_target` function

# Let's crop just wrist area
crop_center = 1600, 800
crop_size = 500, 1000
scale = 0.25
crop_dict = {'crop_center': crop_center, 'crop_size': crop_size}
train_transform = get_transform(augmentation=True, crop_dict=crop_dict, scale=scale)

boneage_dataset = BoneAgeDataset(bone_age_frame=bone_age_frame, root=image_root,
                                 transform=train_transform, target_transform=normalize_target)
nimages = 12
plot_radiographs(boneage_dataset, nimages)

## Evaluate Bone Age model on test data

In [ ]:
crop_center = 1040, 800
h, w = 2000, 1500
scale = 0.25
input_shape = (1, int(h * scale), int(w * scale))
crop_dict = {'crop_center': crop_center, 'crop_size': (h, w)}

# Test dataset
test_frame = pd.read_csv(DATA_PATH / 'test.csv') # ground truth
test_root =  DATA_PATH / 'test'

test_transform = get_transform(augmentation=False, crop_dict=crop_dict, scale=scale)
test_dataset = BoneAgeDataset(bone_age_frame=test_frame, root=test_root, transform=test_transform,
                              target_transform=normalize_target,
                              model_type='age')
data_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=0)

In [ ]:
# Load pretrained model
prev_ckpt = MODELS_DIR / 'bone_age.epoch36-err0.059.pth'
checkpoint = convert_checkpoint(prev_ckpt, {'input_shape': input_shape, 'model_type': 'age'})
model = m46.from_ckpt(checkpoint)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

runner = SupervisedRunner(
    input_key='image', output_key='preds',
    input_target_key='label', device=device
)
predictions = runner.predict_loader(
    model, data_loader,
    verbose=True
)

In [ ]:
# Plot some predictions
predicted_labels = 120 * (1 + predictions.flatten())
true_labels = test_frame['boneage'].array
mae = np.abs(true_labels - predicted_labels).mean()
print(f'Mean absolute error {mae:0.2f} months.', )

nimages = 12
plot_radiographs(test_dataset, nimages, predictions=predicted_labels)

In [ ]:
test_frame['boneage'].array

## Evaluate gender model

In [ ]:
crop_center = 1040, 800
h, w = 2000, 1500
scale = 0.25
input_shape = (1, int(h * scale), int(w * scale))
crop_dict = {'crop_center': crop_center, 'crop_size': (h, w)}

# Test dataset
test_frame = pd.read_csv(DATA_PATH / 'test.csv') # ground truth
test_root =  DATA_PATH / 'test'

test_transform = get_transform(augmentation=False, crop_dict=crop_dict, scale=scale)
test_dataset = BoneAgeDataset(bone_age_frame=test_frame, root=test_root, transform=test_transform,
                              target_transform=None, # do not normalize target
                              model_type='gender')
data_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=0)

In [ ]:
# Load pretrained model
prev_ckpt = MODELS_DIR / 'bone_gender.epoch49-err0.114.pth'
checkpoint = convert_checkpoint(prev_ckpt, {'input_shape': input_shape, 'model_type': 'gender'})
model = m46.from_ckpt(checkpoint)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

output_key = 'probs'
runner = SupervisedRunner(
    input_key='image', output_key=output_key,
    input_target_key='label', device=device
)
predictions = runner.predict_loader(
    model, data_loader,
    verbose=True
)

In [ ]:
predicted_labels = predictions.flatten().round()
true_labels = test_frame['male'].array * 1
accuracy = (true_labels == predicted_labels).mean()
print(f'Mean accuracy {accuracy:0.2f}.')